In [1]:
import ast
import json
%run data_plot.ipynb
%run metrics.ipynb
%run discrete_genetic_algorithm.ipynb
%run ant_colony_optimization.ipynb
%run dijkstras_algorithm.ipynb
%run simulated_annealing.ipynb

In [2]:
def parse_map_data(results):
    """
    parse_map_data(results)
    converts routes data to geographic routes that can be represented on a map.
    this is done by checking the sequence of x y coordinates against their corresponding
    geographic coordinates defined in the plotting module.
    results: the algorithm search results that contains the best solution to the destination
    and all other possible routes.
    returns: the coordinates used to create the map representation
    """
    # init dictionary with the structure expected for map visualization
    coordinates = {'best_route': {'sensor_id': [],
                              'coordinates': []}, 
               'possible_routes': []
              }
    # convert the string containing the list of the best path to type list
    possible_paths = [ast.literal_eval(paths) for paths in results['possible_paths'].keys()]
    # get the geographic coordinates for all nodes in possible paths
    for path in possible_paths:
        route_coords = []
        possible_path_dest_idx = len(path) - 1
        for idx, sensor in enumerate(path):
            if idx != 0 and idx != possible_path_dest_idx:
                # duplicate coordinates for all nodes but for the first and last
                route_coords.append(geographic_coords[str(sensor)])
                route_coords.append(geographic_coords[str(sensor)])
            else:
                route_coords.append(geographic_coords[str(sensor)])
        coordinates['possible_routes'].append(route_coords)
    # add best path last so that an visual it appears on top
    best_path = ast.literal_eval(list(results['best_path'].keys())[0])
    best_path_dest_idx = len(best_path) - 1
    for idx, sensor in enumerate(best_path):
        # include sensor IDs for the best path except for the destination (won't need pin marker)
        if idx != best_path_dest_idx:
            coordinates['best_route']['sensor_id'].append(sensor)
        if idx != 0 and idx != best_path_dest_idx:
            # duplicate coordinates for all nodes but for the first and last
            coordinates['best_route']['coordinates'].append(geographic_coords[str(sensor)])
            coordinates['best_route']['coordinates'].append(geographic_coords[str(sensor)])
        else:
            coordinates['best_route']['coordinates'].append(geographic_coords[str(sensor)])

    return coordinates

In [3]:
def save_solution(solution, method):
    """
    save_solution(solution, method)
    saves the best solution found by each algorithm in a solutions json file.
    the solutions file has a format like...
    {"source node": "Node-5", 
    "routing path": "(Node-2, 1 Mbps), (Node-1, 4Mbps), (BS-2, 2 Mbps)",
    "end-to-end transmission rate": "1 Mbps"}
    solution: the best solution to a base station from a sensor
    method: the method that was used to get that best solution.
    returns: nothing
    """
    # fetch existing solutions
    existing_solutions = load_solutions()
    '''
    if solution for a sensor using the method specified exists, 
    update the dict with a new solution, otherwise create the solution
    '''
    if method in existing_solutions.keys():
        existing_solutions[method][solution[0]['source node']] = solution[0]
    else:
        existing_solutions[method] = {}
        existing_solutions[method][solution[0]['source node']] = solution[0]
    # write the updated solutions to the solutions file
    json.dump(existing_solutions, open( f"solutions.json", 'w' ) )

    return

In [4]:
def load_solutions():
    """
    load_solutions()
    it loads solutions that already exist since new solutions are appended to the json
    object before being added written on the file.
    returns: the json object read from the file
    """
    # fetch existing best solutions from the solutions file
    with open('solutions.json', 'r') as file:
        try:
            solutions = json.load(file)
        except Exception as e:
            # if the solutions file is empty, initialize and empty dict to be used
            solutions = {}

    return solutions

In [5]:
def find_optimal_route(origin, method, **kwargs):
    """
    find_optimal_route(origin, method, **kwargs)
    the main function of this assessment. It searches for best route 
    from a node to either of the 2 base stations. The search is done using
    the algorithm specified as one of the arguments. Supported algorithms 
    include: 'discrete genetic algorithm', 'ant colony optimization', 
    'dijkstras algorithm' and 'simulated annealing'. Each of the algorithms,
    return a best solutions dictionary that is stored in the solutions file,
    and the results dictionary which is used for visualization on a map
    origin: the node whose best path to a base station you are seeking
    method: the algorithm you want to use to perform the search
    **kwargs: any additional arguments you want to pass in for
    optimization of the various algoithms
    Expected **kwargs and their defults:
    discrete genetic algorithm: pop_length=100000, generations=10, mutation_rate=0.1, crossover_type='single-point', selection_limit=20
    ant colony optimization: Q=1, alpha = 0.6, evaporate=0.45, ants_count=20, max_iterations=10
    dijkstras algorithm: None
    simulated annealing: init_temp=100, final_temp=0.1, cooling_rate=0.95
    More details on the **kwargs provided in the algorithms' respective modules
    returns: nothing
    """
    # run the algorithm passed in as a parameter and fetch the visualization results together with the solutions file data
    if method == "discrete genetic algorithm":
        results, solutions = discrete_genetic_algorithm(origin, **kwargs)
    elif method == "dijkstras algorithm":
        results, solutions = dijkstra_algorithm(origin)
    elif method == "ant colony optimization":
        results, solutions = ant_colony_optimization(origin, **kwargs)
    elif method == "simulated annealing":
        results, solutions = simulated_annealing(origin, **kwargs)
    else:
        return "unknown algorithm"
    # save the best path in the solutions file
    save_solution(solutions, method)
    # parse the plotting results and call the map visualization function
    #coordinates = parse_map_data(results)
    #plot_route(coordinates)

    return

In [6]:
#find_optimal_route(43, 'discrete genetic algorithm', pop_length=100000)
#find_optimal_route(1, 'ant colony optimization')
#find_optimal_route(101, 'simulated annealing')

In [7]:
def run_all():
    """
    run_all()
    executes the find_optimal_route() function for all sensors using all algorithms
    for purposes of storing the best solutions for each in the solutions file.
    returns: nothing
    """
    # algorithms to be executed
    methods = ['simulated annealing'] #['discrete genetic algorithm', 'ant colony optimization', 'dijkstras algorithm', 'simulated annealing']
    # all sensors except the 2 base stations
    sensors = list(x_y_data.keys())[41:-2]#list(x_y_data.keys())[:-2]
    # for each method, find the best path for each sensor
    for method in methods:
        for sensor in sensors:
            # ignore situations where no best paths will be found. This is typical with DGA and it will kill the function
            try:
                print(f"Sensor {sensor}")
                find_optimal_route(sensor, method)
            except:
                pass        

    return

In [ ]:
run_all()

Sensor 42
Sensor 43
Sensor 44


In [ ]:
def get_algorithm_time_metrics():
    """
    get_algorithm_time_metrics()
    this function compares the performance of the various algorithms 
    in terms of execution time and plots them on a bar graph. This is done
    by setting a timer at the beginning of execution of an algorithm and stopping
    it at the end of the execution. The duration is stored in a dictionary against
    the algorithm name as a key in the stop_execution() function. These functions
    are found in the found in the metrics module. The durations are plotted on a
    bar graph for visualization purposes.
    returns: nothing
    """
    # choose any random node to start from with the exception of the base stations
    dest_nodes = [x_y_base_station_1[0], x_y_base_station_2[0]]
    start_node = choice(np.array([i for i in x_y_data.keys() if i not in dest_nodes]), 1, replace=False)[0]
    # list of algorithms whose time performance will be compared
    methods = ['discrete genetic algorithm', 'ant colony optimization', 'dijkstras algorithm', 'simulated annealing']
    for method in methods:
        print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
        print(f"executing {method}...")
        # start execution timer
        start_execution()
        # run the algorithm
        find_optimal_route(start_node, method)
        # stop the timer and record the results
        stop_execution(method)
        print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    # plot the recorded results
    plot_algorithm_execution_time()

    return

In [ ]:
#get_algorithm_time_metrics()